## Fake News Classifier Using LSTM



In [1]:
import pandas as pd

In [3]:
df=pd.read_csv('/Users/rahulkosamkar/Documents/Data_Science/Deep Learning/Fake-New-LSTM-master/train.csv')

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
###Drop Nan Values
df=df.dropna()


In [7]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [8]:
## Get the Dependent features
y=df['label']

In [10]:
X.shape

(18285, 4)

In [11]:
y.shape

(18285,)

In [12]:
import tensorflow as tf

In [14]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [15]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [16]:
messages=X.copy()

In [17]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [18]:
messages.reset_index(inplace=True)

In [19]:
import nltk
import re
from nltk.corpus import stopwords

In [20]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rahulkosamkar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [22]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [23]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[97, 1243, 3915, 4912, 903, 3291, 4548, 1905, 1309, 4244],
 [2481, 3995, 3250, 3613, 4971, 591, 3886],
 [3688, 596, 1152, 1484],
 [4652, 3172, 1741, 4437, 4203, 3034],
 [680, 4971, 2575, 4732, 316, 4672, 4971, 1122, 4922, 4529],
 [3410,
  4660,
  4185,
  1891,
  1033,
  4274,
  1596,
  3479,
  770,
  1953,
  486,
  2527,
  3530,
  3095,
  3886],
 [546, 2462, 2312, 2199, 1372, 2447, 2929, 791, 1071, 4704, 2944],
 [1259, 21, 1552, 3900, 1496, 3460, 4274, 4922, 1071, 4704, 2944],
 [977, 268, 3499, 4866, 3410, 1893, 2270, 1083, 4274, 4682],
 [1258, 2373, 1233, 982, 1494, 1144, 3629, 1520],
 [1750, 2355, 3930, 3601, 3169, 3259, 4724, 853, 4303, 841, 3581],
 [4437, 1898, 903, 1893, 4274, 1496],
 [3595, 4160, 3306, 4944, 4593, 3234, 3708, 4310, 4274],
 [2015, 2992, 1768, 623, 4116, 687, 2161, 1071, 4704, 2944],
 [773, 2303, 4333, 1681, 4391, 1071, 4704, 2944],
 [4167, 2700, 1890, 309, 4684, 2114, 2769, 2915, 2726, 2637],
 [1456, 982, 3995],
 [2200, 817, 26, 4401, 4274, 471, 4179, 3886],
 [22

### Embedding Representation

In [24]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1905 1309 4244]
 [   0    0    0 ... 4971  591 3886]
 [   0    0    0 ...  596 1152 1484]
 ...
 [   0    0    0 ... 1071 4704 2944]
 [   0    0    0 ... 3400 3649 3477]
 [   0    0    0 ... 4472 4702  398]]


In [25]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   97,
       1243, 3915, 4912,  903, 3291, 4548, 1905, 1309, 4244], dtype=int32)

In [26]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [27]:
len(embedded_docs),y.shape


(18285, (18285,))

In [28]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [29]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [31]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 8s 33ms/step - loss: 0.3427 - accuracy: 0.8413 - val_loss: 0.2048 - val_accuracy: 0.9153
Epoch 2/10
192/192 [==============================] - 7s 36ms/step - loss: 0.1403 - accuracy: 0.9448 - val_loss: 0.1959 - val_accuracy: 0.9191
Epoch 3/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0934 - accuracy: 0.9645 - val_loss: 0.2182 - val_accuracy: 0.9181
Epoch 4/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0693 - accuracy: 0.9757 - val_loss: 0.2834 - val_accuracy: 0.9104
Epoch 5/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0420 - accuracy: 0.9857 - val_loss: 0.3430 - val_accuracy: 0.9138
Epoch 6/10
192/192 [==============================] - 6s 31ms/step - loss: 0.0211 - accuracy: 0.9931 - val_loss: 0.3789 - val_accuracy: 0.9158
Epoch 7/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0136 - accuracy: 0.9958 - val_loss: 0.4218 - val_accuracy: 0.9118

### Adding Dropout 

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [39]:
y_pred=model.predict(X_test)
y_pred = np.round(y_pred).astype(int)

In [34]:
from sklearn.metrics import confusion_matrix

In [41]:
confusion_matrix(y_test,y_pred)

array([[3068,  351],
       [ 177, 2439]])

In [42]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9125103562551782